In [1]:
from google.colab import files
import io
import pandas as pd

In [2]:
print("Upload all 7 CSV files (orders.csv, delivery_performance.csv, customer_feedback.csv, routes_distance.csv, vehicle_fleet.csv, warehouse_inventory.csv, cost_breakdown.csv)")
uploaded = files.upload()

Upload all 7 CSV files (orders.csv, delivery_performance.csv, customer_feedback.csv, routes_distance.csv, vehicle_fleet.csv, warehouse_inventory.csv, cost_breakdown.csv)


Saving cost_breakdown.csv to cost_breakdown (1).csv
Saving customer_feedback.csv to customer_feedback (1).csv
Saving delivery_performance.csv to delivery_performance (1).csv
Saving orders.csv to orders (1).csv
Saving routes_distance.csv to routes_distance (1).csv
Saving vehicle_fleet.csv to vehicle_fleet (1).csv
Saving warehouse_inventory.csv to warehouse_inventory (1).csv


In [3]:
def read_if_present(name_variants):
    for n in name_variants:
        if n in uploaded:
            return pd.read_csv(io.BytesIO(uploaded[n]))
    return None

orders = read_if_present(["orders.csv","Orders.csv","orders.CSV"])
delivery = read_if_present(["delivery_performance.csv","delivery.csv","delivery_performance.CSV"])
feedback = read_if_present(["customer_feedback.csv","customer_feedback.csv","customer_feedback.CSV"])
routes = read_if_present(["routes_distance.csv","routes_distance.CSV","routes.csv"])
fleet = read_if_present(["vehicle_fleet.csv","vehicle_fleet.CSV","vehicle_fleet.csv"])
warehouse = read_if_present(["warehouse_inventory.csv","warehouse_inventory.CSV","warehouse_inventory.csv"])
costs = read_if_present(["cost_breakdown.csv","cost_breakdown.CSV","cost_breakdown.csv"])

dfs = {
    "orders": orders,
    "delivery_performance": delivery,
    "customer_feedback": feedback,
    "routes_distance": routes,
    "vehicle_fleet": fleet,
    "warehouse_inventory": warehouse,
    "cost_breakdown": costs
}

for name, df in dfs.items():
    print(f"\n=== {name.upper()} ===")
    if df is None:
        print("File not found (please re-upload with correct name).")
        continue
    # show shape and sample columns
    print("Shape:", df.shape)
    display(df.head(3))
    print("Columns:", df.columns.tolist())
    print("Missing values (top 10):")
    print(df.isnull().sum().sort_values(ascending=False).head(10))



=== ORDERS ===
File not found (please re-upload with correct name).

=== DELIVERY_PERFORMANCE ===
File not found (please re-upload with correct name).

=== CUSTOMER_FEEDBACK ===
File not found (please re-upload with correct name).

=== ROUTES_DISTANCE ===
File not found (please re-upload with correct name).

=== VEHICLE_FLEET ===
File not found (please re-upload with correct name).

=== WAREHOUSE_INVENTORY ===
File not found (please re-upload with correct name).

=== COST_BREAKDOWN ===
File not found (please re-upload with correct name).


In [4]:
# --- Load datasets ---
orders = pd.read_csv("orders.csv")
delivery = pd.read_csv("delivery_performance.csv")
feedback = pd.read_csv("customer_feedback.csv")

In [5]:
# --- Clean column names ---
for df in [orders, delivery, feedback]:
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()

In [6]:
# --- Merge main datasets ---
merged = (orders
          .merge(delivery, on="order_id", how="left", suffixes=('_order', '_delivery'))
          .merge(feedback, on="order_id", how="left", suffixes=('', '_feedback')))

In [7]:
# --- Compute delay (actual - promised) ---
merged["delivery_delay_days"] = merged["actual_delivery_days"] - merged["promised_delivery_days"]

# --- Compute combined rating ---
merged["avg_rating"] = merged[["customer_rating", "rating"]].mean(axis=1)

# --- Compute delivery metrics ---
merged["is_late"] = merged["delivery_delay_days"] > 0
merged["avg_delay_days"] = merged["delivery_delay_days"].mean()
merged["median_delay_days"] = merged["delivery_delay_days"].median()
merged["percent_late_deliveries"] = (merged["is_late"].mean()) * 100

In [8]:
# --- Display results ---
print("✅ Data merged successfully!")
print("Average Customer Rating:", round(merged["avg_rating"].mean(), 2))
print("Average Delay (days):", round(merged['delivery_delay_days'].mean(), 2))
print("Median Delay (days):", round(merged['delivery_delay_days'].median(), 2))
print("% of Late Deliveries:", round((merged['delivery_delay_days'] > 0).mean() * 100, 2))
print("Number of Issue Categories:", merged["issue_category"].nunique())
print("Total Orders:", merged["order_id"].nunique())


✅ Data merged successfully!
Average Customer Rating: 3.64
Average Delay (days): 1.08
Median Delay (days): 0.0
% of Late Deliveries: 35.0
Number of Issue Categories: 4
Total Orders: 200


In [9]:
# --- Save final file for Streamlit ---
merged.to_csv("merged_customer_experience.csv", index=False)
print("\n✅ Saved: merged_customer_experience.csv")



✅ Saved: merged_customer_experience.csv


In [12]:
# --- AI MODEL: At-Risk Customer Prediction ---

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

In [13]:
# Download sentiment lexicon
nltk.download("vader_lexicon")

# Compute sentiment score if feedback_text exists
if "feedback_text" in merged.columns:
    sia = SentimentIntensityAnalyzer()
    merged["sentiment_score"] = merged["feedback_text"].astype(str).apply(lambda x: sia.polarity_scores(x)["compound"])
else:
    merged["sentiment_score"] = 0.0

# Define at-risk condition
merged["risk_score"] = (
    (merged["customer_rating"] < 3).astype(int)
    + (merged["delivery_delay_days"] > 2).astype(int)
    + (merged["sentiment_score"] < -0.05).astype(int)
    + (merged["would_recommend"].astype(str).str.lower() == "no").astype(int)
)

merged["at_risk"] = (merged["risk_score"] >= 2).astype(int)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [14]:
# Encode categorical variables
for col in ["customer_segment", "priority", "product_category"]:
    if col in merged.columns:
        le = LabelEncoder()
        merged[col] = le.fit_transform(merged[col].astype(str))

# Prepare features
features = [
    "customer_rating", "delivery_delay_days", "order_value_inr",
    "delivery_cost_inr", "sentiment_score",
    "customer_segment", "priority", "product_category"
]
X = merged[features].fillna(0)
y = merged["at_risk"]

In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=42)

In [16]:
# Evaluate
print("\n✅ Model Trained Successfully!")
print(classification_report(y_test, model.predict(X_test)))


✅ Model Trained Successfully!
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        35
           1       0.80      0.80      0.80         5

    accuracy                           0.95        40
   macro avg       0.89      0.89      0.89        40
weighted avg       0.95      0.95      0.95        40



In [17]:
# Save trained model
joblib.dump(model, "risk_model.pkl")
print("\n💾 Model saved as risk_model.pkl")

# Save merged data for dashboard
merged.to_csv("merged_customer_experience.csv", index=False)
print("✅ Cleaned and enriched dataset saved: merged_customer_experience.csv")



💾 Model saved as risk_model.pkl
✅ Cleaned and enriched dataset saved: merged_customer_experience.csv


In [19]:
# Run in Colab
import pandas as pd

# load files if not already loaded
try:
    orders
except NameError:
    orders = pd.read_csv("orders.csv")
try:
    merged
except NameError:
    merged = pd.read_csv("merged_customer_experience.csv")

# Normalize column names (safe)
for df in [orders, merged]:
    df.columns = df.columns.str.strip()

# 1) Show unique values in original orders.csv for candidate columns
for col in ["customer_segment", "priority", "product_category"]:
    if col in orders.columns:
        print(f"\n--- ORIGINAL orders.csv: {col} unique values ---")
        print(orders[col].value_counts(dropna=False).head(50))
    else:
        print(f"\n(original) Column {col} not present in orders.csv")

# 2) Show unique values in merged file (the numeric codes)
for col in ["customer_segment", "priority", "product_category"]:
    if col in merged.columns:
        print(f"\n--- MERGED file: {col} unique values ---")
        print(merged[col].value_counts(dropna=False).head(50))
    else:
        print(f"\n(merged) Column {col} not present in merged file")

# 3) Try auto-mapping using Order_ID if both files contain it
if "order_id" in orders.columns and "order_id" in merged.columns:
    print("\nAttempting to infer mappings by joining on order_id...")
    join = merged[["order_id", "customer_segment", "priority", "product_category"]].merge(
        orders[["order_id", "customer_segment", "priority", "product_category"]],
        on="order_id", how="left", suffixes=("_merged", "_orig")
    )

    for col in ["customer_segment", "priority", "product_category"]:
        merged_col = f"{col}_merged"
        orig_col = f"{col}_orig"
        if merged_col in join.columns and orig_col in join.columns:
            map_df = join[[merged_col, orig_col]].drop_duplicates().dropna()
            if not map_df.empty:
                mapping = map_df.set_index(merged_col)[orig_col].to_dict()
                print(f"\nInferred mapping for {col} (merged_code -> original_label):")
                print(mapping)
            else:
                print(f"\nNo mapping rows found for {col} via join.")
        else:
            print(f"\nColumns for {col} missing in join.")

    # Save the join to inspect manually if needed
    join.head(50).to_csv("join_preview_for_mapping.csv", index=False)
    print("\nSaved a preview of join (first 50 rows) to 'join_preview_for_mapping.csv'.")
else:
    print("\norder_id not present in both files — cannot do auto join mapping.")


--- ORIGINAL orders.csv: customer_segment unique values ---
customer_segment
SMB           81
Enterprise    60
Individual    59
Name: count, dtype: int64

--- ORIGINAL orders.csv: priority unique values ---
priority
Standard    84
Economy     70
Express     46
Name: count, dtype: int64

--- ORIGINAL orders.csv: product_category unique values ---
product_category
Fashion            34
Books              31
Home Goods         30
Electronics        29
Food & Beverage    29
Industrial         27
Healthcare         20
Name: count, dtype: int64

--- MERGED file: customer_segment unique values ---
customer_segment
2    81
0    60
1    59
Name: count, dtype: int64

--- MERGED file: priority unique values ---
priority
2    84
0    70
1    46
Name: count, dtype: int64

--- MERGED file: product_category unique values ---
product_category
2    34
0    31
5    30
1    29
3    29
6    27
4    20
Name: count, dtype: int64

Attempting to infer mappings by joining on order_id...

Inferred mapping for 